## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

In [0]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Load tensorflow

In [0]:
#!pip3 install -U tensorflow ==2.0 --quiet

In [0]:
import tensorflow as tf

In [5]:
tf.__version__

'2.0.0'

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('/content/drive/My Drive/Great Learning Residency 6.2 neural networks/prices.csv')

### Check all columns in the dataset

In [8]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

In [9]:
data.shape

(851264, 7)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


### Drop columns `date` and  `symbol`

In [0]:
data=data.drop(['date','symbol'],axis=1)

In [12]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
data['volume']=data['volume']/1000000

In [0]:
data_df=data.head(1000)

In [15]:
data_df

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086
...,...,...,...,...,...
995,63.310001,63.590000,63.240002,63.639999,2.1332
996,27.160000,26.990000,26.680000,27.299999,1.9824
997,28.320000,28.770000,28.010000,28.809999,37.1528
998,44.000000,44.799999,43.750000,44.810001,6.5686


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
p=data_df.drop('volume', axis = 1)
q=data_df['volume']

In [0]:
X_train, X_test, y_train, y_test = train_test_split(p, q, test_size=0.30, random_state=1)

In [19]:
X_train.shape

(700, 4)

In [20]:
X_test.shape

(300, 4)

#### Convert Training and Test Data to numpy float32 arrays


In [21]:
X_train.dtypes

open     float64
close    float64
low      float64
high     float64
dtype: object

In [22]:
y_train.dtypes

dtype('float64')

In [0]:
import numpy as np

In [0]:
X_train = np.array(X_train).astype('float32')
y_train = np.array(y_train).astype('float32')
X_test = np.array(X_test).astype('float32')
y_test = np.array(y_test).astype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer

In [0]:
transformer = Normalizer()
X_train = transformer.fit_transform(X_train)

In [0]:
X_test=transformer.transform(X_test)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
w = tf.zeros(shape=(4,1))  
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w) # matrix multiplication between x and w
    y = tf.add(xw_matmul, b)    # add bias
    
    return y

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [54]:
#Train for 100 Steps
for i in range(100):
    
    w, b = train(X_train, y_train, w, b, learning_rate=0.01)
    print('Current Loss on iteration', i, 
          loss(y_train, prediction(X_train, w, b)).numpy())

Current Loss on iteration 0 223.49039
Current Loss on iteration 1 221.28825
Current Loss on iteration 2 219.259
Current Loss on iteration 3 217.38876
Current Loss on iteration 4 215.66518
Current Loss on iteration 5 214.07668
Current Loss on iteration 6 212.61269
Current Loss on iteration 7 211.2635
Current Loss on iteration 8 210.02014
Current Loss on iteration 9 208.87439
Current Loss on iteration 10 207.81828
Current Loss on iteration 11 206.84483
Current Loss on iteration 12 205.94785
Current Loss on iteration 13 205.12125
Current Loss on iteration 14 204.35938
Current Loss on iteration 15 203.65736
Current Loss on iteration 16 203.01028
Current Loss on iteration 17 202.4139
Current Loss on iteration 18 201.86417
Current Loss on iteration 19 201.35777
Current Loss on iteration 20 200.89096
Current Loss on iteration 21 200.46075
Current Loss on iteration 22 200.06422
Current Loss on iteration 23 199.69885
Current Loss on iteration 24 199.36218
Current Loss on iteration 25 199.05173


### Get the shapes and values of W and b

In [57]:
#Check Weights and Bias
print('Weights:\n', w.numpy())
print('Bias:\n',b.numpy())

Weights:
 [[1.3549174]
 [1.3598168]
 [1.3410656]
 [1.3713948]]
Bias:
 [2.7137554]


### Model Prediction on 1st Examples in Test Dataset

In [60]:
prediction(X_test[0:1],w,b)

<tf.Tensor: id=5893139, shape=(1, 1), dtype=float32, numpy=array([[5.427437]], dtype=float32)>

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris=pd.read_csv('/content/drive/My Drive/Great Learning Residency 6.2 neural networks/Iris.csv')

In [78]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
iris_df=pd.get_dummies(iris)

In [80]:
iris_df.head(10)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0
5,6,5.4,3.9,1.7,0.4,1,0,0
6,7,4.6,3.4,1.4,0.3,1,0,0
7,8,5.0,3.4,1.5,0.2,1,0,0
8,9,4.4,2.9,1.4,0.2,1,0,0
9,10,4.9,3.1,1.5,0.1,1,0,0


### Splitting the data into feature set and target set

In [0]:
x=iris_df.drop(['Id','Species_Iris-setosa','Species_Iris-versicolor','Species_Iris-virginica'], axis = 1)
y=iris_df.iloc[0:,5:]

In [105]:
x

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [106]:
y

,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
145,0,0,1
146,0,0,1
147,0,0,1
148,0,0,1


In [0]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=1)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()
#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(1, input_shape=(4,)))

#Normalize the data
model.add(tf.keras.layers.BatchNormalization())

#Add Dense Layer which provides 3 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3, activation='softmax'))

#Comile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])



### Model Training 

In [117]:
model.fit(X_train, y_train, 
          validation_data=(X_test, y_test), 
          epochs=100,
          batch_size=X_train.shape[0])

Train on 105 samples, validate on 45 samples
Epoch 1/100
105/105 [==============================] - 0s 950us/sample - loss: 0.4903 - accuracy: 0.7524 - val_loss: 0.5542 - val_accuracy: 0.7111
Epoch 2/100
105/105 [==============================] - 0s 56us/sample - loss: 0.4898 - accuracy: 0.7524 - val_loss: 0.5537 - val_accuracy: 0.7111
Epoch 3/100
105/105 [==============================] - 0s 45us/sample - loss: 0.4893 - accuracy: 0.7524 - val_loss: 0.5531 - val_accuracy: 0.7111
Epoch 4/100
105/105 [==============================] - 0s 63us/sample - loss: 0.4888 - accuracy: 0.7524 - val_loss: 0.5526 - val_accuracy: 0.7111
Epoch 5/100
105/105 [==============================] - 0s 46us/sample - loss: 0.4884 - accuracy: 0.7524 - val_loss: 0.5521 - val_accuracy: 0.7111
Epoch 6/100
105/105 [==============================] - 0s 43us/sample - loss: 0.4879 - accuracy: 0.7524 - val_loss: 0.5516 - val_accuracy: 0.7111
Epoch 7/100
105/105 [==============================] - 0s 38us/sample - loss: 

### Model Prediction

In [112]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 1)                 5         
_________________________________________________________________
batch_normalization_6 (Batch (None, 1)                 4         
_________________________________________________________________
dense_10 (Dense)             (None, 3)                 6         
Total params: 15
Trainable params: 13
Non-trainable params: 2
_________________________________________________________________


In [114]:
pred = model.predict(X_test)
pred

array([[0.87496   , 0.11151931, 0.01352074],
       [0.6155405 , 0.28250834, 0.10195114],
       [0.15254195, 0.35009626, 0.49736178],
       [0.873466  , 0.11272191, 0.013812  ],
       [0.0049456 , 0.09939636, 0.8956581 ],
       [0.10649861, 0.32039258, 0.57310885],
       [0.05286441, 0.25673282, 0.69040275],
       [0.80586714, 0.1643882 , 0.02974461],
       [0.8721388 , 0.1137881 , 0.01407305],
       [0.02213463, 0.18534163, 0.79252374],
       [0.24815112, 0.37626243, 0.3755865 ],
       [0.8237378 , 0.15124235, 0.02501982],
       [0.01674167, 0.16576098, 0.8174973 ],
       [0.13581519, 0.3410646 , 0.5231202 ],
       [0.15866745, 0.35298708, 0.48834547],
       [0.906032  , 0.08587667, 0.00809122],
       [0.29325953, 0.37805602, 0.32868442],
       [0.1704561 , 0.3579859 , 0.47155803],
       [0.8612571 , 0.12244935, 0.01629349],
       [0.9039923 , 0.08759759, 0.00841009],
       [0.21368706, 0.37090793, 0.415405  ],
       [0.17828822, 0.36092526, 0.46078655],
       [0.

In [128]:
for i in pred:
  predicted_num = np.argmax(i)
  print(predicted_num)

0
0
2
0
2
2
2
0
0
2
1
0
2
2
2
0
1
2
0
0
2
2
2
0
2
1
0
0
2
2
2
2
1
2
2
0
1
0
2
2
2
0
2
2
2


### Save the Model

In [0]:
model.save('Iris.h5')


### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?